In [45]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig,pipeline
import os
import pandas as pd
from langchain.chains import ConversationChain

from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.memory import (
    CombinedMemory,
    ConversationBufferMemory,
    ConversationSummaryMemory,
)

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline


In [46]:
df = pd.read_csv("Salaries.csv")
df.head()

,rank,discipline,phd,service,sex,salary
0,Prof,B,56,49,Male,186960
1,Prof,A,12,6,Male,93000
2,Prof,A,23,20,Male,110515
3,Prof,A,40,31,Male,131205
4,Prof,B,20,18,Male,104800


In [47]:
df.name = 'Salaries.csv'
df.name

'Salaries.csv'

In [70]:
!gpustat

gpu-server           Tue Feb 27 15:47:46 2024  495.29.05
[0] Quadro RTX 5000  | 39'C,   0 % |  6760 / 16122 MB | user(6743M) gdm(9M) gdm(3M)


In [49]:
gpu_to_use = 0
device = torch.device(f'cuda:{gpu_to_use}' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [50]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5",
                                          load_in_4bit = True
                                         )


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [51]:
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5",
                                             load_in_4bit = True)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [52]:
pipe = pipeline("text-generation", 
                model=model, 
                tokenizer=tokenizer,
                top_k = 1,
                do_sample = True,
                temperature = 0.2,
                max_new_tokens = 200,
                eos_token_id = [10897,63,4686,10252],
                
               )
llm = HuggingFacePipeline(pipeline=pipe)

In [3]:
# print(llm.invoke("Hi"))

In [73]:
# prompt = """You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
# When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# ### Rules:
# 1) print values above the bars of bar plot.
# 2) the output should be phrased with respect to question.
# 3) Use seaborn or plotly along with matplotlib.
# 4) do not print values above the bar if it gets too congested.
# 5) only give the code without explanation and comments.
# 6) Always import necessary libraries and load the dataset.
# 7) Plot only when told.
# 8) Show does not always means you have to plot 
#     # Follow the example below
#     User: Show me columns of the dataset
#     Assistant:
#     import pandas as pd
#     df = pd.read_csv("Dataset.csv")
#     print(df.columns)

# # You should strictly follow the above rule

# Here's the scoop on the dataset "{dataset_name}" boasting records featuring diverse columns:
# {dataset_sample}

# # Follow the example below
# User: Give me salary distribution across all sex
# Assistant: 
# import pandas as pd
# df = pd.read_csv("Salaries.csv")
# salary_distribution = df.groupby('sex')['salary'].mean()
# colors = ['skyblue', 'lightcoral']
# salary_distribution.plot(kind='bar', color=colors)
# plt.title('Salary Distribution Across Sexes')
# plt.xlabel('Sex')
# plt.ylabel('Average Salary')
# plt.show()

# # Assist user

# User: {question}
# Assistant:
# ```python
# """

# _DEFAULT_TEMPLATE = """You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
# When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# # You should strictly follow the above rule

# Here's the scoop on the dataset "{dataset_name}" boasting records featuring diverse columns:
# {dataset_sample}


# Past conversations:
# {history}
# # Assist user

# User: {input}
# Assistant:
# ```python
# """

In [55]:
instruct_prompt =  """You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "{dataset_name}" boasting records featuring diverse columns
{dataset_sample}


"""


input_prompt = """
Past conversations:
{history}
# Assist user

User: {input}
Assistant:
```python
"""



In [56]:
instruct_prompt = PromptTemplate.from_template(instruct_prompt).format(dataset_name=df.name,dataset_sample=df.head(5))


In [57]:
print(instruct_prompt)

You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "Salaries.csv" boasting records featuring diverse columns:
   rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800



In [58]:
final_prompt = instruct_prompt + input_prompt
print(final_prompt)

You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "Salaries.csv" boasting records featuring diverse columns:
   rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800

Past conversations:
{history}
# Assist user

User: {input}
Assistant:
```python



In [63]:
final_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=instruct_prompt + input_prompt,
)


In [65]:
print(final_prompt.template)

You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "Salaries.csv" boasting records featuring diverse columns:
   rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800

Past conversations:
{history}
# Assist user

User: {input}
Assistant:
```python



In [59]:

# conv_memory_dname = ConversationBufferMemory(
#     memory_key="dataset_name", input_key="input"
# )

# conv_memory_dsample = ConversationBufferMemory(
#     memory_key="dataset_sample", input_key="input"
# )
# memory = CombinedMemory(memories=[conv_memory_dname, conv_memory_dsample])


In [66]:
memory = ConversationBufferMemory()

In [67]:
conversation = ConversationChain(llm=llm, 
                                 verbose=True,
                                 memory=memory,
                                 prompt=final_prompt)



In [68]:
print(conversation.run(input="for each gender and discipline plot average salaries"))

/home/user/miniconda3/envs/mayur2/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:10897 for open-end generation.
/home/user/miniconda3/envs/mayur2/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')




> Entering new ConversationChain chain...
Prompt after formatting:
You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "Salaries.csv" boasting records featuring diverse columns:
   rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800

Past conversations:

# Assist user

User: for each gender and discipline plot average salaries
Assistant:
```python


> Finished chain.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('Salaries.csv')

# Calculate t

In [69]:
print(conversation.run(input="using plotly"))

Setting `pad_token_id` to `eos_token_id`:10897 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:
You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "Salaries.csv" boasting records featuring diverse columns:
   rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800

Past conversations:
Human: for each gender and discipline plot average salaries
AI: import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('Salaries.csv')

# Calculate the average salary for each gender and discipline
avg

In [71]:
print(conversation.run(input="add a new column named new_salary by dividing salary by 1000"))

Setting `pad_token_id` to `eos_token_id`:10897 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:
You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "Salaries.csv" boasting records featuring diverse columns:
   rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800

Past conversations:
Human: for each gender and discipline plot average salaries
AI: import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('Salaries.csv')

# Calculate the average salary for each gender and discipline
avg

In [72]:
print(conversation.run(input="find avg of the new column"))

Setting `pad_token_id` to `eos_token_id`:10897 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:
You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "Salaries.csv" boasting records featuring diverse columns:
   rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800

Past conversations:
Human: for each gender and discipline plot average salaries
AI: import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('Salaries.csv')

# Calculate the average salary for each gender and discipline
avg

In [74]:
print(conversation.run(input="show me first 5 rows"))

Setting `pad_token_id` to `eos_token_id`:10897 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:
You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

# You should strictly follow the above rule

Here's the scoop on the dataset "Salaries.csv" boasting records featuring diverse columns:
   rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800

Past conversations:
Human: for each gender and discipline plot average salaries
AI: import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('Salaries.csv')

# Calculate the average salary for each gender and discipline
avg